<a href="https://colab.research.google.com/github/JoeOlang/Drafts/blob/master/Vision/submission_cats_vs_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a short demo on image classification using CNN using data from Kaggle. 
For more info on the dataset see this [link.](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data) <br>
Note: The Dataset may change over time.

In [1]:
import cv2
import numpy as np
import pandas as pd

import os


from sklearn.model_selection import train_test_split
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img

import warnings
warnings.filterwarnings('ignore')

In [2]:
test_dir = '/content/drive/MyDrive/Colab Notebooks/Computer Vision/Cats&Dogs/test'
test_images = ['/content/drive/MyDrive/Colab Notebooks/Computer Vision/Cats&Dogs/test/{}'.format(i) for i in os.listdir(test_dir)]

In [3]:
# Resize images to 150 by 150 and use RGB (3) Colour Model
nrows = 150
ncolumns = 150
channels = 3    # colour-RGB. channel, change to 1 for grayscale

In [4]:
# read - resize function
def process_img(list_of_imgs):

    X = []  #Rsized images
    y = []  #Labels
    id = []

    for image in list_of_imgs:
        X.append(cv2.resize(cv2.imread(image, cv2.IMREAD_COLOR), (nrows, ncolumns), interpolation = cv2.INTER_CUBIC))   #read image
        # get labels
        if 'dog' in image:
            y.append(1)
        elif 'cat' in image:
            y.append(0)
        id.append(image)

    return X, y, id

In [5]:
# load saved model i.e. model_keras

from tensorflow import keras    
model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Computer Vision/Cats&Dogs/tl_model_keras.h5')

In [6]:
len(test_images)

12500

In [7]:
X_test, y_test, ids = process_img(test_images)     #Y_test will be empty since we dont have labels for these
x = np.array(X_test)
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
len(X_test)

12500

In [9]:
def unpack(prediction):
    x = prediction.flatten()
    x = x.tolist()
    x = x[0]
    return x

In [11]:
# Use a loop to go over the test cases and make predictions
columns = 5
i = 0
text_labels = []
scores = []

#plt.figure(figsize=(30,20))
for batch in test_datagen.flow(x, batch_size=1):
    pred = model.predict(batch)
    if pred > 0.5:
        text_labels.append('dog')
    else:
        text_labels.append('cat')
    
    score = unpack(pred)
    scores.append(score)

    #plt.subplot(5 / columns + 1, columns, i + 1)
    #plt.title('This has been predicted as a: '  + text_labels[i])
    #imgplot = plt.imshow(batch[0])
    i += 1
    if i % 12500 == 0:
        break

#plt.show()

In [13]:
len(text_labels)

12500

In [16]:
type(text_labels)

list

In [44]:
# create sub dataframe

df = pd.DataFrame(list(zip(ids, text_labels, scores)),
               columns =['id', 'item', 'label'])

In [45]:
import re
def extract_number(x):
    y = re.findall(r'\d+', x)
    if len(y) == 0:
        z = ''
    else:
        z = int(y[0])
    return z

In [46]:
df['id'] = df['id'].apply(lambda x: extract_number(x))

In [48]:
expdf = df

In [49]:
del expdf['item']

In [50]:
expdf = expdf.sort_values(by=['id'], ascending=True)

In [52]:
expdf.to_csv('/content/drive/MyDrive/Colab Notebooks/Computer Vision/Cats&Dogs/submission_file1.csv', index=False)